[Dataset found here](https://www.jyu.fi/hytk/fi/laitokset/mutku/en/research/projects2/past-projects/coe/materials/emotion/soundtracks/Index).

# Kernel SVM to Predict Song Mood

## Import Libraries

In [1]:
import pandas as pd
from pathlib import Path
import librosa
import IPython.display as ipd
import numpy as np
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings("ignore") #librosa.load raises warning with .mp3 files

## Import Dependent Variable

In [2]:
dataset = pd.read_excel(io='set2/mean_ratings_set2.xls')
dataset.head()

,Number,valence,energy,tension,anger,fear,happy,sad,tender,beauty,liking,TARGET,Soundtrack,Index in Set 1
0,1,2.5625,7.7292,8.3750,6.3881,4.4030,1.2090,1.7463,1.0149,4.7066,4.9661,ANGER_HIGH,Lethal weapon 3,227
1,2,2.5000,8.2083,8.5625,5.7612,5.9254,1.0896,1.5970,1.1045,4.5709,4.7915,ANGER_HIGH,The Rainmaker,124
2,3,2.9167,8.2083,8.1667,6.3134,4.0597,1.3134,1.5522,1.0149,4.4488,5.0614,ANGER_HIGH,The Alien Trilogy,125
3,4,2.8750,6.7500,7.9792,5.4179,4.5075,1.1940,2.1045,1.0597,3.9969,4.3546,ANGER_HIGH,Cape Fear,168
4,5,3.0833,6.8333,7.5000,6.1343,4.2388,1.0746,2.9552,1.0746,5.0765,5.3549,ANGER_HIGH,The Fifth Element,239


### Generalize Target Labels

In [3]:
dataset['TARGET'].unique()

array(['ANGER_HIGH', 'ANGER_MODERATE', 'FEAR_HIGH', 'FEAR_MODERATE',
       'HAPPY_HIGH', 'HAPPY_MODERATE', 'SAD_HIGH', 'SAD_MODERATE',
       'TENDER_HIGH', 'TENDER_MODERATE', 'VALENCE POS HIGH',
       'VALENCE POS MODERATE', 'VALENCE NEG MODERATE', 'VALENCE NEG HIGH',
       'ENERGY POS HIGH', 'ENERGY POS MODERATE', 'ENERGY NEG MODERATE',
       'ENERGY NEG HIGH', 'TENSION POS HIGH', 'TENSION POS MODERATE',
       'TENSION NEG MODERATE', 'TENSION NEG HIGH'], dtype=object)

In [4]:
moods = ['ANGER','FEAR','HAPPY','SAD','TENDER','VALENCE','ENERGY','TENSION']
for mood in moods:
    dataset.TARGET=np.where(dataset.TARGET.str.startswith(mood),mood,dataset.TARGET)

In [5]:
dataset['TARGET']

0        ANGER
1        ANGER
2        ANGER
3        ANGER
4        ANGER
        ...   
105    TENSION
106    TENSION
107    TENSION
108    TENSION
109    TENSION
Name: TARGET, Length: 110, dtype: object

### Encode Labels

In [6]:
y = dataset['TARGET']
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)
y

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7,
       7, 7, 7, 7, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6])

## Import Audio

In [7]:
DIRECTORY = Path('set2/mp3')

audio_files = sorted([fp for fp in DIRECTORY.glob('*.mp3')])
audio = [librosa.load(i) for i in audio_files]

### Extract Features

In [69]:
def extract_features(signal):
    return [
#         librosa.feature.zero_crossing_rate(signal)[0, 0],
        librosa.feature.spectral_centroid(signal)[0, 0],
#         librosa.feature.mfcc(signal)[0,0],
        librosa.feature.spectral_rolloff(signal)[0,0],
        librosa.feature.spectral_contrast(signal)[0,0],
        librosa.feature.spectral_flatness(signal)[0,0],
#         librosa.feature.rms(signal)[0,0],
#         librosa.feature.tonnetz(signal)[0,0]
    ]

In [70]:
X = pd.DataFrame([extract_features(audio[x][0]) for x in range(len(audio))])

### Split into Test and Train

In [71]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 0)

### Feature Scale

In [72]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Train Kernel SVM Model

### Dimensionality Reduction with Principal Component Analysis

In [93]:
from sklearn.decomposition import PCA
pca = PCA(n_components = 2)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

### Use Radial Basis Function to Train KSVM

In [94]:
from sklearn.svm import SVC
classifier = SVC(kernel = 'rbf')
classifier.fit(X_train, y_train)

SVC()

## Evaluate

### Confusion Matrix

In [95]:
from sklearn.metrics import confusion_matrix, accuracy_score
y_pred = classifier.predict(X_test)
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[0 3 0 0 0 0 0 1]
 [0 2 0 0 0 0 3 0]
 [0 2 0 0 0 0 0 1]
 [0 2 0 0 0 0 0 1]
 [0 1 0 0 0 0 0 0]
 [0 1 0 0 0 1 0 0]
 [0 0 0 0 0 1 3 1]
 [0 1 0 0 0 0 0 4]]


0.35714285714285715

### Use K-Fold Cross Validation to Evaluate Accuracy

In [96]:
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = classifier,
                            X = X_train,
                            y = y_train,
                            cv = 10)
print("Accuracy: {:.2f}%".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

Accuracy: 25.83%
Standard Deviation: 12.02 %
